In [ ]:
from typing import Optional

import abc
import numpy as np
import math
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/train_dataset_DCS.zip -d train_dataset_DCS

Archive:  /content/drive/My Drive/train_dataset_DCS.zip
  inflating: train_dataset_DCS/supermarket_val.tsv  
  inflating: train_dataset_DCS/supermarket_val_target.tsv  
  inflating: train_dataset_DCS/cosmetic_train.tsv  
  inflating: train_dataset_DCS/cosmetic_val.tsv  
  inflating: train_dataset_DCS/cosmetic_val_target.tsv  
  inflating: train_dataset_DCS/supermarket_train.tsv  


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
# !pip install wandb

In [ ]:
# !wandb login

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM, MobileBertForMaskedLM
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch
import matplotlib.pyplot as plt
from torch.optim import AdamW
import os

In [ ]:
def tsv_to_txt(out_path, *in_paths):
    dfs = []

    for path in in_paths:
        df = pd.read_csv(path, delimiter='\t')[['receipt_id', 'device_id', 'item_id']]
        dfs.append(df)

    extended_df = pd.concat(dfs, ignore_index=True)
    extended_df['device_id'] = extended_df['device_id'].astype(str)
    extended_df['item_id'] = extended_df['item_id'].astype(str)

    grouped_df = extended_df.groupby('receipt_id').agg({'device_id': 'first', 'item_id': lambda x: ' '.join(x)}).reset_index()

    mask_file_path = out_path.replace(".txt", "_mask.txt")

    # Файл без масок
    with open(out_path, 'w') as no_mask_file, open(mask_file_path, 'w') as mask_file:
        for _, row in grouped_df.iterrows():
            items = row['item_id'].split()

            for i in range(len(items)):
                # записываем оригинальную строку в файл без масок
                no_mask_line = f"R{row['receipt_id']} D{row['device_id']} {' '.join(['I' + item for item in items])}\n"
                no_mask_file.write(no_mask_line)

                # создаем и записываем строку с маской
                masked_items = items.copy()
                masked_items[i] = "[MASK]"
                mask_line = f"R{row['receipt_id']} D{row['device_id']} {' '.join(['I' + item if item != '[MASK]' else item for item in masked_items])}\n"
                mask_file.write(mask_line)

In [ ]:
path = "/content/train_dataset_DCS/"

In [ ]:
# Пример использования:
tsv_to_txt(path+'data.txt', path+'cosmetic_train.tsv', path+'cosmetic_val.tsv', path+'supermarket_train.tsv', path+'supermarket_val.tsv')

#Пример
#9127014611 352398080037759 [MASK] 104817
#9127014611 352398080037759 106012 [MASK]

#9127014611 352398080037759 106012 104817
#9127014611 352398080037759 106012 104817

In [ ]:
with open(path+'data.txt', 'r') as file:
    data = file.readlines()

with open(path+'data_mask.txt', 'r') as file:
    data_mask = file.readlines()

assert len(data) == len(data_mask)

In [ ]:
# !pip install datasets

In [ ]:
# from datasets import Dataset, DatasetDict

In [ ]:
# dataset = Dataset.from_dict({"text": data_mask, "labels": data})

In [79]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
# dataset.push_to_hub("parseny/DCS_dataset")

In [86]:
!pip install transformers

In [87]:
!git clone https://github.com/huggingface/transformers
%cd transformers

Cloning into 'transformers'...
remote: Enumerating objects: 159056, done.
remote: Counting objects: 100% (1507/1507), done.
remote: Compressing objects: 100% (831/831), done.
remote: Total 159056 (delta 894), reused 1087 (delta 611), pack-reused 157549
Receiving objects: 100% (159056/159056), 160.09 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (119100/119100), done.
/content/transformers


In [88]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ffz_xo00
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ffz_xo00
  Resolved https://github.com/huggingface/transformers to commit 95b374952dc27d8511541d6f5a4e22c9ec11fb24
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [90]:
%cd content/transformers/examples/pytorch/language-modeling/

[Errno 2] No such file or directory: 'content/transformers/examples/pytorch/language-modeling/'
/content/transformers


In [82]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [93]:
!pip install transformers[torch]

  Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)


In [94]:
!python /content/transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path distil \
    --dataset_name parseny/DCS_dataset \
    --output_dir "./test" \
    --report_to all \
    --fp16 \
    --mlm_probability 0.15 \
    --do_train \
    --do_eval \
    --per_device_train_batch_size 64 \
    --num_train_epochs 3 \
    --overwrite_output_dir True \
    --push_to_hub \
    --hub_model_id parseny/BertRec \

2023-09-10 00:58:04.536030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/transformers/examples/pytorch/language-modeling/run_mlm.py", line 684, in <module>
    main()
  File "/content/transformers/examples/pytorch/language-modeling/run_mlm.py", line 256, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "/usr/local/lib/python3.10/dist-packages/transformers/hf_argparser.py", line 347, in parse_args_into_dataclasses
    raise ValueError(f"Some specified arguments are not used by the HfArgumentParser: {remaining_args}")
ValueError: Some specified arguments are not used by the HfArgumentParser: ['\\']


In [66]:
train_data_mask, test_data_mask, train_data, test_data = train_test_split(data_mask[:114500], data[:114500], test_size=0.2, random_state=42)

In [53]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [67]:
class SimpleTokenizer:
    def __init__(self):
        self.special_tokens = ['[MASK]', '[PAD]']
        self.token2id = {token: idx for idx, token in enumerate(self.special_tokens)}
        self.id2token = {idx: token for token, idx in self.token2id.items()}

    def tokenize(self, text):
        return text.split()  # Разбиваем текст на слова

    def encode(self, text, add_special_tokens=True, return_tensors=None, padding='max_length', max_length=None):
        tokens = self.tokenize(text)
        input_ids = [self.token_to_id(token) for token in tokens]

        if padding == 'max_length' and len(input_ids) < max_length:
            input_ids += [self.token_to_id('[PAD]') for _ in range(max_length - len(input_ids))]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long)
        }

    def token_to_id(self, token):
        if token not in self.token2id:
            # Расширяем словарь для новых токенов
            new_id = len(self.token2id)
            self.token2id[token] = new_id
            self.id2token[new_id] = token
        return self.token2id[token]

    def id_to_token(self, token_id):
        return self.id2token[token_id]

In [68]:
tokenizer = SimpleTokenizer()

In [69]:
class MaskedDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        label = self.labels[idx]

        encoding = tokenizer.encode(item, return_tensors='pt', padding='max_length', max_length=100)
        label_encoding = tokenizer.encode(label, return_tensors='pt', padding='max_length', max_length=100)['input_ids']

        # Заменяем все метки, кроме [MASK], значением -100
        mask_idx = tokenizer.token_to_id('[MASK]')
        label_encoding = torch.where(encoding['input_ids'] == mask_idx, label_encoding, torch.tensor(-100))

        encoding['input_ids'] = encoding['input_ids'].squeeze()
        encoding['attention_mask'] = (encoding['input_ids'] != tokenizer.token_to_id('[PAD]')).long()
        encoding['labels'] = label_encoding.squeeze()

        return encoding

In [70]:
train_dataset = MaskedDataset(train_data_mask, train_data)
test_dataset = MaskedDataset(test_data_mask, test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4)

In [ ]:
# train_data_mask, test_data_mask, train_data, test_data = train_test_split(data_mask, data, test_size=0.1, random_state=42)

In [ ]:
# train_data_mask[1]

In [ ]:
# ds = DatasetDict({"train": np., "validation": test_dataset})

In [ ]:
# ds.push_to_hub("parseny/DCS_dataset")

In [71]:
from transformers import MobileBertForMaskedLM

In [72]:
model = MobileBertForMaskedLM.from_pretrained("google/mobilebert-uncased")
model.train()
torch.cuda.empty_cache()

In [73]:
optimizer = AdamW(model.parameters(), lr=1e-4)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [75]:
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0

    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

        torch.cuda.empty_cache()
    return total_loss / len(dataloader)

In [76]:
def validate_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        loop = tqdm(dataloader, leave=True, desc="Validation")
        for batch in loop:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()

            torch.cuda.empty_cache()
    return total_loss / len(dataloader)

In [64]:
def train_model(model, train_loader, val_loader, optimizer, device, save_path, epochs=3):
    train_losses = []
    val_losses = []

    best_val_loss = float('inf')

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train_loss = train_epoch(model, train_loader, optimizer, device)
        val_loss = validate_epoch(model, val_loader, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            print(f"New best validation loss: {val_loss:.4f}. Saving model...")
            best_val_loss = val_loss

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            model.save_pretrained(save_path)

        torch.cuda.empty_cache()

In [95]:
save_path = '/content/drive/My Drive/bert'
torch.cuda.empty_cache()
train_model(model, train_loader, test_loader, optimizer, device, save_path)

Epoch 1/3


  0%|          | 0/1432 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 4/1432 [00:03<23:38,  1.01it/s, loss=15.3]


KeyboardInterrupt: ignored

In [ ]:
def predict_masked_token(model, tokenizer, input_str, device):
    inputs = tokenizer.encode(input_str, return_tensors='pt', padding='max_length', max_length=100)

    inputs['input_ids'] = inputs['input_ids'].unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(inputs['input_ids'])
        predictions = outputs.logits[0]

    mask_index = torch.where(inputs['input_ids'][0] == tokenizer.token_to_id('[MASK]'))[0].item()

    predicted_token_id = torch.argmax(predictions[0, mask_index]).item()

    predicted_token = tokenizer.id_to_token(predicted_token_id)

    return predicted_token

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

input_str = "R9193846208 D352398080550058 I101426 [MASK]"
predicted_token = predict_masked_token(model, tokenizer, input_str, device)
print(f"Predicted token for [MASK] is: {predicted_token}")

Predicted token for [MASK] is: [MASK]


In [50]:
# Получение всех токенов
tokens = list(tokenizer.token2id.keys())
print(tokens)

['[MASK]', '[PAD]', 'R9127014611', 'D352398080037759', 'I104817', 'R9651412570', 'D352398080464094', 'I111969', 'I115870', 'R9133486996', 'D352398081864565', 'I109201', 'I100282', 'I114926', 'I105015', 'I117135', 'R9193846208', 'D352398080550058', 'I101426']
